In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
class Linear(nn.Module):
    def __init__(self):
        super(Linear, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(64, 32),
            nn.Tanh()
        )
    
    def forward(self, z):
        output = self.model(z)
        return output

In [5]:
linear = Linear()
print(linear)

Linear(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): BatchNorm1d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): Tanh()
  )
)


In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # 主路径
        self.main = nn.Sequential(
            nn.Linear(in_channels, out_channels, stride),
            nn.BatchNorm1d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Linear(out_channels, out_channels),
            nn.BatchNorm1d(out_channels)
        )

        # 残差路径（shortcut或skip connection）
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Linear(in_channels, out_channels, stride),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        return nn.LeakyReLU(inplace=True)(self.main(x) + self.shortcut(x))

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.model = nn.Sequential(
            ResidualBlock(100, 128),
            ResidualBlock(128, 64),
            ResidualBlock(64, 32),
            nn.Tanh()
        )

    def forward(self, z):
        output = self.model(z)
        return output

In [9]:
x = torch.arange(-5, 5, 0.001).to(device)
y = torch.tanh(x).to(device)